#IMPORTING LIBRARIES AND MOUNTING DRIVE



In [ ]:
!pip install biopython
!pip install silence_tensorflow

In [2]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
from Bio.PDB import *

path='./'

import sys
sys.path.append("/home/liang/code/acdc-nn/acdc_nn")
import util
import nn

#REPRODUCING PAPER RESULTS

In the following cells we reproduce ACDC-NN results on test and myoglobin datasets. 
The network has been trained in transfer learning on s2648 and Ivankov, therefore the cv sets have been generated taking into account the similarity between proteins (Similarity < 25%). First of all we load the proper weights based on the cross-validation, then we generate protein structures, create the input for the network in the appropriate form and also generate the reverse mutation. The reverse structures were created using MODELLER.
Finally we predict the $\Delta \Delta G$ with ACDC-NN. 


#test prediction with ACDC-NN* (2 structures)

In [8]:
p53_dir=pd.read_csv(path+'test_TS.mut', sep=',',header=None)
p53_inv=p53_dir.copy()
p53_inv.loc[:,1]=p53_inv.loc[:,1].apply(lambda x : x[-1]+x[1:-1]+x[0])
p53_inv.loc[:,2]=p53_inv.loc[:,2].apply(lambda x : -x)


pred_p53_dir=list()
pred_p53_inv=list()

# building the specific model with cv weights

path_weights=path+"weights_cv/Weights_PostTL_CV_5"
num_H=[32,16]
d=0.2
ACDC_NN=nn.build_acdc_3d(num_H,d,25)[0]
ACDC_NN.load_weights(path_weights)

#p53 prediction using both structures

for (protein_dir,protein_inv,mut_dir,mut_inv) in zip(list(p53_dir[0]),list(p53_inv[0]),list(p53_dir[1]),list(p53_inv[1])):

# information processing
# get structure and other information for the direct protein

  prof_path_dir ='/home/liang/psi/testprof/' + protein_dir +'.prof' 
  pdb_path_dir= '/home/liang/testpbd/' + protein_dir +'.pdb'
  chain_dir = protein_dir[-1]

  structure_dir, pchain_dir, seq_dir, d_seq2pdb_dir, d_pdb2seq_dir  = util.pdb2info(pdb_path_dir, chain_dir)
  prof_dir = util.getProfile(prof_path_dir)

  kvar_dir=(mut_dir[0],d_pdb2seq_dir[mut_dir[1:-1]],mut_dir[-1])
  kvar_pdb_dir=(mut_dir[0],mut_dir[1:-1],mut_dir[-1])

  dist_neigh_3d_dir= util.get_neigh_ps(kvar_pdb_dir,5,d_seq2pdb_dir,pchain_dir) 
  list_dist_neigh_3d_dir = dist_neigh_3d_dir[kvar_dir]

  # extracting features
  codif_dir=util.getMutCod(mut_dir)
  all_profile_dir = util.Unified_prof(kvar_dir[1],prof_dir,seq_dir, list_dist_neigh_3d_dir)

  #dir 
  To_predict_dir=pd.DataFrame([*codif_dir,*all_profile_dir,*np.zeros(600-len(all_profile_dir))]).T

  # information processing
  # get structure and other information for the inverse protein

  prof_path_inv =path + 'profiles/' + protein_inv +'.prof.gz' 
  pdb_path_inv= path + 'pdbs/' + protein_inv[:-1] +'_' + mut_dir + '_' +'.pdb.gz'
  chain_inv = protein_inv[-1]

  # information processing
  # get structure and other information for the inverse protein

  structure_inv, pchain_inv, seq_inv, d_seq2pdb_inv, d_pdb2seq_inv  = util.pdb2info(pdb_path_inv, chain_inv)
  prof_inv = util.getProfile(prof_path_inv)

  kvar_inv=(mut_inv[0],d_pdb2seq_inv[mut_inv[1:-1]],mut_inv[-1])
  kvar_pdb_inv=(mut_inv[0],mut_inv[1:-1],mut_inv[-1])

  dist_neigh_3d_inv= util.get_neigh_ps(kvar_pdb_inv,5,d_seq2pdb_inv,pchain_inv) 
  list_dist_neigh_3d_inv = dist_neigh_3d_inv[kvar_inv]

  # extracting features
  codif_inv=util.getMutCod(mut_inv)
  all_profile_inv = util.Unified_prof(kvar_inv[1],prof_inv,seq_inv, list_dist_neigh_3d_inv)

  #inv
  To_predict_inv=pd.DataFrame([*codif_inv,*all_profile_inv,*np.zeros(600-len(all_profile_inv))]).T

  # Making input in the proper shape
   
  Xm_d, X1D_d, X3D_d = nn.mkInp_3d(np.asarray(To_predict_dir).astype(np.float32),500)
  Xm_i, X1D_i, X3D_i = nn.mkInp_3d(np.asarray(To_predict_inv).astype(np.float32),500)  

  #predict dir
  prediction_dir=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
  pred_p53_dir.append(prediction_dir[0][0][0])

  #predict inv
  prediction_inv=ACDC_NN.predict([X3D_i, X1D_i, Xm_i , X3D_d, X1D_d, Xm_d])
  pred_p53_inv.append(prediction_inv[0][0][0])    


KeyError: '5'

MEASURE OF ACDC-NN* PERFORMANCE ON P53

P53 direct 

In [ ]:
print('pearson direct :',np.round(np.corrcoef(pred_p53_dir,p53_dir[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(pred_p53_dir,p53_dir[2])),2))

pearson direct : 0.61
rmse :  1.69


P53 inverse

In [ ]:
print('pearson inverse :',np.round(np.corrcoef(pred_p53_inv,p53_inv[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(pred_p53_inv,p53_inv[2])),2))

pearson inverse : 0.61
rmse :  1.69


Antisimmetry

In [ ]:
print('r-dir-inv :',np.corrcoef(pred_p53_dir,pred_p53_inv)[0][1])
print('bias : ',np.round(util.bias(pred_p53_dir,pred_p53_inv),2))

r-dir-inv : -1.0
bias :  0.0


#P53 prediction with ACDC-NN (1 structures)

In [29]:
p53_pred_dir=list()
p53_pred_inv=list()

p53_dir=pd.read_csv('/home/liang/test_ddg.csv', sep=',',header= None)
p53_inv=p53_dir.copy()
print(p53_inv.columns)
p53_inv.loc[:,1]=p53_inv.loc[:,1].apply(lambda x : x[-1]+x[1:-1]+x[0])
p53_inv.loc[:,2]=p53_inv.loc[:,2].apply(lambda x : -x)


# building the specific model with cv weights

path_weights=path+"weights_cv/Weights_PostTL_CV_5"
num_H=[32,16]
d=0.2
ACDC_NN=nn.build_acdc_3d(num_H,d,25)[0]
ACDC_NN.load_weights(path_weights)

#Ssym dir prediction
for protein,mut in zip(list(p53_dir[0]),list(p53_dir[1])):

  prof_path ='/home/liang/code/acdc-nn/tests/profiles/' + protein +'.prof' 
  pdb_path= '/home/liang/code/acdc-nn/tests/structures/' + protein +'.pdb'
  chain = 'A'

  # information processing
  # get structure and other information
  structure, pchain, seq, d_seq2pdb, d_pdb2seq  = util.pdb2info(pdb_path, chain)
  prof = util.getProfile(prof_path)

  kvar=(mut[0],d_pdb2seq[mut[1:-1]],mut[-1])
  kvar_pdb=(mut[0],mut[1:-1],mut[-1])

  dist_neigh_3d= util.get_neigh_ps(kvar_pdb,5,d_seq2pdb,pchain) 
  list_dist_neigh_3d = dist_neigh_3d[kvar]

  # extracting features
  codif=util.getMutCod(mut)
  all_profile = util.Unified_prof(kvar[1],prof,seq, list_dist_neigh_3d)
  
  #dir 
  To_predict_dir=pd.DataFrame([*codif,*all_profile,*np.zeros(600-len(all_profile))]).T

  #inv (dir)
  To_predict_inv=To_predict_dir.copy()
  To_predict_inv.iloc[:,:20]=To_predict_inv.iloc[:,:20].replace([1.0,-1.0],[-1.0,1.0])
  
  # Making input in the proper shape 
  Xm_d, X1D_d, X3D_d = nn.mkInp(np.asarray(To_predict_dir).astype(np.float32),500)
  Xm_i, X1D_i, X3D_i = nn.mkInp(np.asarray(To_predict_inv).astype(np.float32),500)  

  #predict
  prediction=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
  p53_pred_dir.append(prediction[0][0][0])

#Ssym inv prediction
for protein,mut in zip(list(p53_inv[0]),list(p53_inv[1])):
  
  mut_dir=mut[-1]+mut[1:-1]+mut[0]
  prof_path ='/home/liang/code/acdc-nn/tests/profiles' + protein +'.prof' 
  pdb_path= '/home/liang/code/acdc-nn/tests/structures' + protein +'.pdb'
  chain = 'A'

  # information processing
  # get structure and other information
  structure, pchain, seq, d_seq2pdb, d_pdb2seq  = util.pdb2info(pdb_path, chain)
  prof = util.getProfile(prof_path)

  kvar=(mut[0],d_pdb2seq[mut[1:-1]],mut[-1])
  kvar_pdb=(mut[0],mut[1:-1],mut[-1])

  dist_neigh_3d= util.get_neigh_ps(kvar_pdb,5,d_seq2pdb,pchain) 
  list_dist_neigh_3d = dist_neigh_3d[kvar]

  # extracting features
  codif=util.getMutCod(mut)
  all_profile = util.Unified_prof(kvar[1],prof,seq, list_dist_neigh_3d)
  
  #dir
  To_predict_dir=pd.DataFrame([*codif,*all_profile,*np.zeros(600-len(all_profile))]).T
  
  #dir (inv)
  To_predict_inv=To_predict_dir.copy()
  To_predict_inv.iloc[:,:20]=To_predict_inv.iloc[:,:20].replace([1.0,-1.0],[-1.0,1.0])
  
  # Making input in the proper shape 
  Xm_d, X1D_d, X3D_d = nn.mkInp(np.asarray(To_predict_dir).astype(np.float32),500)
  Xm_i, X1D_i, X3D_i = nn.mkInp(np.asarray(To_predict_inv).astype(np.float32),500)  

  #predict
  prediction=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
  p53_pred_inv.append(prediction[0][0][0])

Int64Index([0, 1, 2], dtype='int64')
Key '241' does not exist in the dictionary.


KeyError: '241'

MEASURE OF ACDC-NN PERFORMANCE ON P53



P53 direct 

In [ ]:
print('pearson direct :',np.round(np.corrcoef(p53_pred_dir,p53_dir[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(p53_pred_dir,p53_dir[2])),2))

pearson direct : 0.62
rmse :  1.67


P53 inverse

In [ ]:
print('pearson direct :',np.round(np.corrcoef(p53_pred_inv,p53_inv[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(p53_pred_inv,p53_inv[2])),2))

pearson direct : 0.61
rmse :  1.72


Antisimmetry

In [ ]:
print('r-dir-inv :',np.round(np.corrcoef(p53_pred_dir,p53_pred_inv)[0][1],2))
print('bias : ',np.round(util.bias(p53_pred_dir,p53_pred_inv),2))

r-dir-inv : -0.99
bias :  -0.01


# Myoglobin prediction with ACDC-NN* (2 structures)



In [ ]:
myo_dir=pd.read_csv(path+'1bz6A_TS_0.mut', sep=' ',header=None)
myo=[[j[0].values[0],j[1].values[0],np.round(np.mean(j[2].values),2)] for i,j in myo_dir.groupby(1)]
myo_dir=pd.DataFrame(myo)

#inv
myo_inv=myo_dir.copy()
myo_inv.loc[:,1]=myo_inv.loc[:,1].apply(lambda x : x[-1]+x[1:-1]+x[0])
myo_inv.loc[:,2]=myo_inv.loc[:,2].apply(lambda x : -x)

pred_myo_dir=list()
pred_myo_inv=list()

# building the specific model with cv weights

path_weights=path+"weights_cv/Weights_PostTL_CV_0"
num_H=[32,16]
d=0.2
ACDC_NN=nn.ACDC(num_H,d,25)[0]
ACDC_NN.load_weights(path_weights)

#p53 prediction using both structures

for (protein_dir,protein_inv,mut_dir,mut_inv) in zip(list(myo_dir[0]),list(myo_inv[0]),list(myo_dir[1]),list(myo_inv[1])):

# information processing
# get structure and other information for the direct protein

  prof_path_dir =path + 'profiles/' + protein_dir +'.prof.gz' 
  pdb_path_dir= path + 'pdbs/' + protein_dir[:-1] +'.pdb.gz'
  chain_dir = protein_dir[-1]

  structure_dir, pchain_dir, seq_dir, d_seq2pdb_dir, d_pdb2seq_dir  = util.pdb2info(pdb_path_dir, chain_dir)
  prof_dir = util.getProfile(prof_path_dir)

  kvar_dir=(mut_dir[0],d_pdb2seq_dir[mut_dir[1:-1]],mut_dir[-1])
  kvar_pdb_dir=(mut_dir[0],mut_dir[1:-1],mut_dir[-1])

  dist_neigh_3d_dir= util.get_neigh_ps(kvar_pdb_dir,5,d_seq2pdb_dir,pchain_dir) 
  list_dist_neigh_3d_dir = dist_neigh_3d_dir[kvar_dir]

  # extracting features
  codif_dir=util.getMutCod(mut_dir)
  all_profile_dir = util.Unified_prof(kvar_dir[1],prof_dir,seq_dir, list_dist_neigh_3d_dir)

  #dir 
  To_predict_dir=pd.DataFrame([*codif_dir,*all_profile_dir,*np.zeros(600-len(all_profile_dir))]).T

  # information processing
  # get structure and other information for the inverse protein

  prof_path_inv =path + 'profiles/' + protein_inv +'.prof.gz' 
  pdb_path_inv= path + 'pdbs/' + protein_inv[:-1] +'_' + mut_dir + '_' +'.pdb.gz'
  chain_inv = protein_inv[-1]

  # information processing
  # get structure and other information for the inverse protein

  structure_inv, pchain_inv, seq_inv, d_seq2pdb_inv, d_pdb2seq_inv  = util.pdb2info(pdb_path_inv, chain_inv)
  prof_inv = util.getProfile(prof_path_inv)

  kvar_inv=(mut_inv[0],d_pdb2seq_inv[mut_inv[1:-1]],mut_inv[-1])
  kvar_pdb_inv=(mut_inv[0],mut_inv[1:-1],mut_inv[-1])

  dist_neigh_3d_inv= util.get_neigh_ps(kvar_pdb_inv,5,d_seq2pdb_inv,pchain_inv) 
  list_dist_neigh_3d_inv = dist_neigh_3d_inv[kvar_inv]

  # extracting features
  codif_inv=util.getMutCod(mut_inv)
  all_profile_inv = util.Unified_prof(kvar_inv[1],prof_inv,seq_inv, list_dist_neigh_3d_inv)

  #inv
  To_predict_inv=pd.DataFrame([*codif_inv,*all_profile_inv,*np.zeros(600-len(all_profile_inv))]).T

  # Making input in the proper shape
   
  Xm_d, X1D_d, X3D_d = nn.mkInp(np.asarray(To_predict_dir).astype(np.float32),500)
  Xm_i, X1D_i, X3D_i = nn.mkInp(np.asarray(To_predict_inv).astype(np.float32),500)  

  #predict dir
  prediction_dir=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
  pred_myo_dir.append(prediction_dir[0][0][0])

  #predict inv
  prediction_inv=ACDC_NN.predict([X3D_i, X1D_i, Xm_i , X3D_d, X1D_d, Xm_d])
  pred_myo_inv.append(prediction_inv[0][0][0])    

MEASURE OF ACDC-NN* PERFORMANCE ON MYO


Myo direct

In [ ]:
print('pearson direct :',np.round(np.corrcoef(pred_myo_dir,myo_dir[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(pred_myo_dir,myo_dir[2])),2))

pearson direct : 0.58
rmse :  0.89


Myo inverse

In [ ]:
print('pearson inverse :',np.round(np.corrcoef(pred_myo_inv,myo_inv[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(pred_myo_inv,myo_inv[2])),2))

pearson inverse : 0.58
rmse :  0.89


Antisimmetry

In [ ]:
print('pearson direct :',np.round(np.corrcoef(pred_myo_dir,pred_myo_inv)[0][1],2))
print('bias : ',np.round(util.bias(pred_myo_dir,pred_myo_inv),2))

pearson direct : -1.0
bias :  0.0


# Myoglobin prediction with ACDC-NN (1 structure)


In [ ]:
myo_dir=pd.read_csv(path+'1bz6A_TS_0.mut', sep=' ',header=None)
myo=[[j[0].values[0],j[1].values[0],np.round(np.mean(j[2].values),2)] for i,j in myo_dir.groupby(1)]
myo_dir=pd.DataFrame(myo)

#inv
myo_inv=myo_dir.copy()
myo_inv.loc[:,1]=myo_inv.loc[:,1].apply(lambda x : x[-1]+x[1:-1]+x[0])
myo_inv.loc[:,2]=myo_inv.loc[:,2].apply(lambda x : -x)

myo_pred_dir=list()
myo_pred_inv=list()

# building the specific model with cv weights

path_weights=path+"weights_cv/Weights_PostTL_CV_0"
num_H=[32,16]
d=0.2
ACDC_NN=nn.ACDC(num_H,d,25)[0]
ACDC_NN.load_weights(path_weights)

#Ssym dir prediction
for protein,mut in zip(list(myo_dir[0]),list(myo_dir[1])):

  prof_path =path + 'profiles/' + protein +'.prof.gz' 
  pdb_path= path + 'pdbs/' + protein[:-1] +'.pdb.gz'
  chain = protein[-1]

  # information processing
  # get structure and other information
  structure, pchain, seq, d_seq2pdb, d_pdb2seq  = util.pdb2info(pdb_path, chain)
  prof = util.getProfile(prof_path)

  kvar=(mut[0],d_pdb2seq[mut[1:-1]],mut[-1])
  kvar_pdb=(mut[0],mut[1:-1],mut[-1])

  dist_neigh_3d= util.get_neigh_ps(kvar_pdb,5,d_seq2pdb,pchain) 
  list_dist_neigh_3d = dist_neigh_3d[kvar]

  # extracting features
  codif=util.getMutCod(mut)
  all_profile = util.Unified_prof(kvar[1],prof,seq, list_dist_neigh_3d)
  
  #dir 
  To_predict_dir=pd.DataFrame([*codif,*all_profile,*np.zeros(600-len(all_profile))]).T

  #inv (dir)
  To_predict_inv=To_predict_dir.copy()
  To_predict_inv.iloc[:,:20]=To_predict_inv.iloc[:,:20].replace([1.0,-1.0],[-1.0,1.0])
  
  # Making input in the proper shape 
  Xm_d, X1D_d, X3D_d = nn.mkInp(np.asarray(To_predict_dir).astype(np.float32),500)
  Xm_i, X1D_i, X3D_i = nn.mkInp(np.asarray(To_predict_inv).astype(np.float32),500)  

  #predict
  prediction=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
  myo_pred_dir.append(prediction[0][0][0])

#Ssym inv prediction
for protein,mut in zip(list(myo_inv[0]),list(myo_inv[1])):
  
  mut_dir=mut[-1]+mut[1:-1]+mut[0]
  prof_path =path + 'profiles/' + protein +'.prof.gz' 
  pdb_path= path + 'pdbs/'  + protein[:-1] +'_' + mut_dir + '_'+'.pdb.gz'
  chain = protein[-1]

  # information processing
  # get structure and other information
  structure, pchain, seq, d_seq2pdb, d_pdb2seq  = util.pdb2info(pdb_path, chain)
  prof = util.getProfile(prof_path)

  kvar=(mut[0],d_pdb2seq[mut[1:-1]],mut[-1])
  kvar_pdb=(mut[0],mut[1:-1],mut[-1])

  dist_neigh_3d= util.get_neigh_ps(kvar_pdb,5,d_seq2pdb,pchain) 
  list_dist_neigh_3d = dist_neigh_3d[kvar]

  # extracting features
  codif=util.getMutCod(mut)
  all_profile = util.Unified_prof(kvar[1],prof,seq, list_dist_neigh_3d)
  
  #dir
  To_predict_dir=pd.DataFrame([*codif,*all_profile,*np.zeros(600-len(all_profile))]).T
  
  #dir (inv)
  To_predict_inv=To_predict_dir.copy()
  To_predict_inv.iloc[:,:20]=To_predict_inv.iloc[:,:20].replace([1.0,-1.0],[-1.0,1.0])
  
  # Making input in the proper shape 
  Xm_d, X1D_d, X3D_d = nn.mkInp(np.asarray(To_predict_dir).astype(np.float32),500)
  Xm_i, X1D_i, X3D_i = nn.mkInp(np.asarray(To_predict_inv).astype(np.float32),500)  

  #predict
  prediction=ACDC_NN.predict([X3D_d, X1D_d, Xm_d , X3D_i, X1D_i, Xm_i])
  myo_pred_inv.append(prediction[0][0][0])

MEASURE OF ACDC-NN PERFORMANCE ON MYO


Myo direct

In [ ]:
print('pearson direct :',np.round(np.corrcoef(myo_pred_dir,myo_dir[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(myo_pred_dir,myo_dir[2])),2))

pearson direct : 0.58
rmse :  0.89


Myo inverse

In [ ]:
print('pearson direct :',np.round(np.corrcoef(myo_pred_inv,myo_inv[2])[0][1],2))
print('rmse : ',np.round(sqrt(mean_squared_error(myo_pred_inv,myo_inv[2])),2))

pearson direct : 0.57
rmse :  0.89


Antisimmetry

In [ ]:
print('r-dir-inv :',np.round(np.corrcoef(myo_pred_dir,myo_pred_inv)[0][1],2))
print('bias : ',np.round(util.bias(myo_pred_dir,myo_pred_inv),2))

r-dir-inv : -0.99
bias :  -0.01
